#### Global Parameters

In [1]:
N_Reconfigurations = 10**1

# NEU Unit (Reconfiguration Map)

## Initialize Modules

###### Imports

In [2]:
# Deep Learning & ML
import tensorflow as tf
import keras as K
from keras import backend as Kback
from keras.models import Sequential
from keras import layers
from keras import utils as np_utils

# General
import numpy as np
import time

# Message Boxes
import tkinter
from tkinter import messagebox

# General Outputs
print('TensorFlow:', tf.__version__)

TensorFlow: 2.1.0


Using TensorFlow backend.


### Define Reconfiguration Unit

1. Shifts $x \in \mathbb{R}^d$ to $x- c$; c trainable.
2. Rescales componentwise with $a * x$,  $a \in \mathbb{R}^d$ trainable.
3. Applies the map $\psi(x)\triangleq e^{\frac1{1-|x|}}I_{\{|x|<1\}}$ component-wise.  
4. Applies transformation $x \mapsto x +b$, $b \in \mathbb{R}^d$ trainable.
5. Applies the diagonalization map to that output: $ \left(x_1,\dots,x_d\right)\mapsto
                \begin{pmatrix}
                x_1 & & 0\\
                &\ddots &\\
                0 & & x_d\\
                \end{pmatrix}.$
6. Applies map $X \mapsto XA$, $A$ is a trainable $d\times d$ matrix.
7. Applies matrix exponential.
8. Multiplies output with result of (1).
9. Re-centers output to $x +c$ where $c$ is as in (1).

In [3]:
class Reconfiguration_unit(tf.keras.layers.Layer):
    def __init__(self, *args, **kwargs):
        super(Reconfiguration_unit, self).__init__(*args, **kwargs)

    def build(self, input_shape):
        self.weight = self.add_weight(shape=[input_shape[1],input_shape[1]], # CURRENTLY NOT A MATRIX FIX THIS!!!
                                    initializer='zeros',
                                    trainable=True)
        self.bias = self.add_weight(shape=input_shape[1:],
                                    initializer='zeros',
                                    trainable=True)
        self.location = self.add_weight(shape=input_shape[1:],
                                    initializer='zeros',
                                    trainable=True)
        self.scale = self.add_weight(shape=input_shape[1:],
                                    initializer='zeros',
                                    trainable=True)
        
        
        
    def call(self, x):
        # 1. Shift and scale data
        x_shift = x - self.location
        
        # 2. Rescale componentwise
        x_mod = tf.math.multiply(x_shift,self.scale)
        
        # 3. Apply bumpy function Component-wise
        x_in_abs = tf.math.abs(x_mod)
        Logic_x_leq1 = tf.math.sign(tf.keras.activations.relu(1-x_in_abs)) # Takes value 1 iff |x|<=1 else 0: since probability of |x|=1 is 0 we should be ok
        x_thresheld = Logic_x_leq1*tf.math.exp(-1/(1-tf.math.pow(x_in_abs,-1))) # Computes bump function at thresholds with previous logic
        
        # 4+5. Apply Shift (In Tangent Space) and diagonalize
        x_out = tf.expand_dims((x_in_abs + self.bias), 1) 
        
        # 6. Multiply by weight matrix (in Tangent Space)
        x_out = (x_out * self.weight) 
        
        # 7. Apply Matrix Exponential
        x_out = tf.linalg.expm(x_out)
        
        # 8. Muliply by output of (1)
        x_out = tf.linalg.matvec(x_out,x_shift)
        
        # 9. Recenter Transformed Data
        x_out = x_out + self.location
        
        # Return Ouput
        return x_out

#### NEU-OLS Layer
This Layer maps $x \mapsto (x,Ax+b)$, where $A \in \operatorname{Mat}_{d\times d}$ and $b \in \mathbb{R}^d$ are both trainable.

In [52]:
class NEU_OLS_layer(tf.keras.layers.Layer):
    def __init__(self, *args, **kwargs):
        super(NEU_OLS_layer, self).__init__(*args, **kwargs)

    # Defines A and b to be trainable
    def build(self, input_shape):
        self.weight = self.add_weight(shape=(1,input_shape[1]),
                                    initializer='ones',
                                    trainable=True)
        self.bias = self.add_weight(shape=(input_shape[0]),
                                    initializer='zeros',
                                    trainable=True)

        super(NEU_OLS_layer, self).build(input_shape)


    def call(self, x):
        # Apply Linear Regression
        x_out = tf.matmul(self.weight,x,transpose_b=True) + self.bias
        # Concatenate map output with input to form  graph!
        x_out = tf.concat([x,tf.transpose(x_out)],-1)
        return x_out

#### Build Reconfiguration

Iniitalize and Build Layers of Reconfiguration

In [57]:
input_layer = tf.keras.Input(shape=[1])
current_layer = NEU_OLS_layer()
current_layer_out = current_layer(input_layer)

In [58]:
for i in range(10):
    current_layer = Reconfiguration_unit()(current_layer_out)

Compile Model

In [59]:
NEU_OLS = tf.keras.Model(inputs=[input_layer], outputs=[current_layer])
NEU_OLS.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_20 (InputLayer)        [(None, 1)]               0         
_________________________________________________________________
neu_ols_layer_18 (NEU_OLS_la (None, 2)                 2         
_________________________________________________________________
reconfiguration_unit_149 (Re (None, 2)                 10        
Total params: 12
Trainable params: 12
Non-trainable params: 0
_________________________________________________________________
